# X-Match Gold FGKM

In this notebook, we have the code to run all the crossmatching between:
- Gold sample and main Gaia table
- Cross match with Bailer Jones distances

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astroquery.gaia import Gaia

from astropy.table import Table
import astropy.coordinates as ac
_ = ac.galactocentric_frame_defaults.set('v4.0') #set the default Astropy Galactocentric frame parameters to the values adopted in Astropy v4.0
import astropy.units as u
from astropy.io import fits

In [2]:
Gaia.login()
# schaini

INFO: Login to gaia TAP server [astroquery.gaia.core]
User: zivezic
Password: ········
OK
INFO: Login to gaia data server [astroquery.gaia.core]
OK


## 1. FGKM base table

In [3]:
# # Getting metadata for the FGKM sample of stars.
# fgkm_table = Gaia.load_table('gaiadr3.gold_sample_fgkm_stars')
# print(fgkm_table, '\n')

# # Looking at which data columns are available to use in this sample.
# for column in fgkm_table.columns:
#     print(column.name)

In [4]:
%%time

# Asynchronous query with Gaia to get the all data from FGKM golden sample
job_all = Gaia.launch_job_async("SELECT * FROM gaiadr3.gold_sample_fgkm_stars")
# Setting for fgkm and checking to make sure we have the data we want.
fgkm = job_all.get_results()
fgkm = fgkm.to_pandas().sort_values('source_id')
fgkm = fgkm.reset_index(drop=True)
fgkm.to_csv("fgkm_sample.csv",index=False)
fgkm

INFO: Query finished. [astroquery.utils.tap.core]
CPU times: user 6min, sys: 5.91 s, total: 6min 6s
Wall time: 10min 13s


,source_id,teff_gspphot,logg_gspphot,mh_gspphot,ag_gspphot,ebpminrp_gspphot,alphafe_gspspec,teff_gspspec,logg_gspspec,mh_gspspec,...,lum_flame,mass_flame,age_flame,evolstage_flame,radius_flame_spec,lum_flame_spec,mass_flame_spec,age_flame_spec,evolstage_flame_spec,spectraltype_esphs
0,8044474553216,4806.233887,4.5114,-0.0697,0.0711,0.0375,NaN,NaN,NaN,NaN,...,0.328582,NaN,NaN,393,NaN,NaN,NaN,NaN,<NA>,
1,12545600306304,5249.593262,4.4557,-0.0848,0.1124,0.0597,NaN,NaN,NaN,NaN,...,0.596999,0.842317,13.142073,349,NaN,NaN,NaN,NaN,<NA>,
2,14401026146688,5172.659180,4.4139,-0.2540,0.1672,0.0892,NaN,NaN,NaN,NaN,...,0.612476,NaN,NaN,352,NaN,NaN,NaN,NaN,<NA>,
3,15637976759168,5741.219238,4.1478,-0.0963,0.2220,0.1191,NaN,NaN,NaN,NaN,...,2.649110,1.061862,8.610679,445,NaN,NaN,NaN,NaN,<NA>,
4,16531329957376,5654.772949,4.3989,-0.1923,0.0009,0.0005,NaN,NaN,NaN,NaN,...,0.979605,0.939067,9.112502,312,NaN,NaN,NaN,NaN,<NA>,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3273036,6917508003777163904,4969.641602,4.5520,-0.0460,0.0688,0.0364,NaN,NaN,NaN,NaN,...,0.330749,0.798405,8.977385,246,NaN,NaN,NaN,NaN,<NA>,
3273037,6917508038136902016,5210.476562,4.4257,0.1178,0.0456,0.0241,NaN,NaN,NaN,NaN,...,0.692772,NaN,NaN,359,NaN,NaN,NaN,NaN,<NA>,
3273038,6917511371031526272,4595.608398,4.5494,-0.0827,0.1113,0.0586,NaN,NaN,NaN,NaN,...,0.205604,0.729048,9.092461,244,NaN,NaN,NaN,NaN,<NA>,
3273039,6917517865022020736,5706.174316,4.3798,-0.4858,0.0047,0.0025,NaN,NaN,NaN,NaN,...,1.060684,0.957715,8.385532,306,NaN,NaN,NaN,NaN,<NA>,


## 2. Basic source parameters table

In [5]:
# source_table = Gaia.load_table('gaiadr3.gaia_source')
# for column in source_table.columns:
#     print(column.name)

In [6]:
%%time

# Crossmatching source id's of FGKM sample with Gaia for ra, dec, galac_l, galac_b, proper motion, proper motion at ra and dec, 
# its error and correlation, radial velocity, radial velocity error (which is specified in Gaia's
# data release as the uncertainty value), mag in g, blue part and red part, colors bp-rp and g-rp
job2 = Gaia.launch_job_async("SELECT source_id, ra, dec, l, b, pm, pmra, pmra_error, pmdec, pmdec_error, pmra_pmdec_corr, radial_velocity, radial_velocity_error, phot_g_mean_mag, phot_bp_mean_mag, phot_rp_mean_mag, bp_rp, g_rp \
FROM gaiadr3.gaia_source AS dr3 \
JOIN gaiadr3.gold_sample_fgkm_stars AS xmatch USING (source_id) ")

# Checking results.
fgkm_new = job2.get_results()
fgkm_new = fgkm_new.to_pandas().sort_values('source_id')
fgkm_new.to_csv("fgkm_source.csv",index=False)
fgkm_new

INFO: Query finished. [astroquery.utils.tap.core]
CPU times: user 5min 28s, sys: 8.67 s, total: 5min 36s
Wall time: 14min 55s


,source_id,ra,dec,l,b,pm,pmra,pmra_error,pmdec,pmdec_error,pmra_pmdec_corr,radial_velocity,radial_velocity_error,phot_g_mean_mag,phot_bp_mean_mag,phot_rp_mean_mag,bp_rp,g_rp
0,8044474553216,44.927666,0.219128,176.645475,-48.802311,14.712925,11.987892,0.033959,-8.529984,0.025886,-0.044210,NaN,NaN,14.947446,15.464661,14.277565,1.187096,0.669881
1,12545600306304,45.234476,0.318152,176.860886,-48.513020,48.337337,0.056719,0.020818,-48.337303,0.015860,0.011556,-48.036938,1.309670,12.194942,12.620842,11.601145,1.019697,0.593798
2,14401026146688,45.047858,0.295074,176.689759,-48.663170,6.636381,6.281287,0.034993,2.141725,0.025406,-0.152498,NaN,NaN,14.904602,15.351336,14.283580,1.067757,0.621022
3,15637976759168,45.194622,0.344149,176.790851,-48.523609,1.950195,1.941703,0.015542,-0.181805,0.013826,0.200773,-34.489674,2.639760,13.135885,13.505640,12.597779,0.907861,0.538106
4,16531329957376,45.141363,0.359598,176.718137,-48.551124,89.935150,-53.789714,0.024427,-72.076330,0.016337,-0.077507,-25.610313,0.362432,10.033792,10.359525,9.538733,0.820792,0.495060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3273036,6917508003777163904,315.132676,-0.285956,48.717997,-28.521277,22.378012,20.849992,0.029277,8.127312,0.017501,-0.046591,-70.833977,2.909547,14.178848,14.655931,13.545896,1.110035,0.632953
3273037,6917508038136902016,315.147027,-0.278247,48.733944,-28.529586,21.632872,-3.938550,0.030909,-21.271317,0.019352,0.025524,28.056076,5.061578,14.647898,15.072821,14.065042,1.007778,0.582855
3273038,6917511371031526272,315.169686,-0.165559,48.857057,-28.490746,11.508323,-10.953868,0.044485,3.529058,0.027088,-0.012659,NaN,NaN,15.320883,15.903053,14.593727,1.309326,0.727156
3273039,6917517865022020736,314.715243,-0.254421,48.504216,-28.147333,14.606207,14.288730,0.018689,3.028776,0.013359,0.161111,-14.030037,2.306355,13.432541,13.744101,12.950566,0.793534,0.481975


## 3. Distance table (from Gaia eDR3)

In [7]:
# # Checking for the Bailer-Jones distance catalog's existence within Gaia.
# distance_table = Gaia.load_table("external.gaiaedr3_distance")
# print(distance_table)

# # Looking at the column metadata for the catalog's table.
# for column in distance_table.columns:
#     print(column.name)

In [8]:
%%time

# Crossmatching with the FGKM sample.
job3 = Gaia.launch_job_async("SELECT source_id, r_med_geo, r_lo_geo, r_hi_geo, r_med_photogeo, r_lo_photogeo, r_hi_photogeo \
FROM external.gaiaedr3_distance \
JOIN gaiadr3.gold_sample_fgkm_stars AS xmatch USING (source_id)")

# Checking results.
fgkm_distance = job3.get_results()
fgkm_distance = fgkm_distance.to_pandas().sort_values('source_id')
fgkm_distance = fgkm_distance.reset_index(drop=True)
fgkm_distance.to_csv("fgkm_distance.csv",index=False)
fgkm_distance

INFO: Query finished. [astroquery.utils.tap.core]
CPU times: user 2min 17s, sys: 2.3 s, total: 2min 20s
Wall time: 6min 7s


,source_id,r_med_geo,r_lo_geo,r_hi_geo,r_med_photogeo,r_lo_photogeo,r_hi_photogeo
0,8044474553216,590.816833,580.010071,601.690918,593.353882,583.349243,603.878113
1,12545600306304,230.104507,228.831650,230.998367,230.060242,229.116577,231.293259
2,14401026146688,785.214478,772.176270,801.768005,786.467041,768.195007,802.981628
3,15637976759168,732.026306,724.801819,739.273132,730.343506,722.221802,738.582642
4,16531329957376,116.741631,116.450081,117.016655,116.666084,116.459892,116.906517
...,...,...,...,...,...,...,...
3273036,6917508003777163904,424.231659,419.788605,429.749115,425.029114,419.317657,429.755829
3273037,6917508038136902016,792.768860,775.370605,809.882507,791.127869,772.299316,808.631470
3273038,6917511371031526272,528.545166,519.096680,538.945618,531.882263,520.089417,542.794556
3273039,6917517865022020736,576.041565,569.238525,581.648071,577.798279,571.414368,583.945740


## 4. SDSS table

In [9]:
# #SDSS
# # Checking for the Bailer-Jones distance catalog's existence within Gaia.
# SDSS_ = Gaia.load_table("external.sdssdr13_photoprimary")
# print(SDSS_)

# # Looking at the column metadata for the catalog's table.
# for column in SDSS_.columns:
#     print(column.name)

In [10]:
# for column in Gaia.load_table("gaiadr3.sdssdr13_join").columns:
# for column in Gaia.load_table("gaiadr3.sdssdr13_neighbourhood").columns:
# for column in Gaia.load_table("gaiadr3.sdssdr13_best_neighbour").columns:
#     print(column.name)

In [11]:
%%time

job5 = Gaia.launch_job_async("SELECT gaia.source_id, xjoin.original_ext_source_id, sdss.psfmag_u, sdss.psfmag_g, sdss.psfmag_r, sdss.psfmag_i, sdss.psfmag_z, sdss.psfmagerr_u, sdss.psfmagerr_g, sdss.psfmagerr_r, sdss.psfmagerr_i, sdss.psfmagerr_z \
FROM external.sdssdr13_photoprimary AS sdss \
JOIN gaiadr3.sdssdr13_best_neighbour AS xjoin \
    ON xjoin.original_ext_source_id = sdss.objid \
JOIN gaiadr3.gold_sample_fgkm_stars AS gaia USING (source_id)")

fgkm_sdss = job5.get_results()
fgkm_sdss = fgkm_sdss.to_pandas().sort_values('source_id')
fgkm_sdss = fgkm_sdss.reset_index(drop=True)
fgkm_sdss.to_csv("fgkm_sdss.csv",index=False)
fgkm_sdss

INFO: Query finished. [astroquery.utils.tap.core]
CPU times: user 1min 7s, sys: 1.52 s, total: 1min 8s
Wall time: 6min 15s


,source_id,original_ext_source_id,psfmag_u,psfmag_g,psfmag_r,psfmag_i,psfmag_z,psfmagerr_u,psfmagerr_g,psfmagerr_r,psfmagerr_i,psfmagerr_z
0,8044474553216,1237666301090136066,17.773451,15.741870,14.89205,15.225330,14.48534,0.015836,0.014353,0.012603,0.002154,0.020897
1,12545600306304,1237666301090267141,14.517120,12.783800,12.15883,11.947370,11.84606,0.029319,0.000382,0.000311,0.000315,0.013419
2,14401026146688,1237666301090136215,17.258909,15.503620,14.87164,15.230740,14.49686,0.014295,0.014339,0.012604,0.002173,0.020892
3,15637976759168,1237666301090201808,15.114560,13.670590,14.77194,17.000500,12.86236,0.022253,0.002956,0.005737,0.095059,0.020055
4,16531329957376,1237666301090201733,12.010040,10.473430,10.01602,9.884631,10.04061,0.000699,0.000130,0.000115,0.000141,0.000320
...,...,...,...,...,...,...,...,...,...,...,...,...
865565,6917507694539517568,1237663457238712813,19.739050,17.118759,15.87243,15.398140,15.11562,0.031586,0.009783,0.008023,0.010035,0.014056
865566,6917508003777163904,1237663457238712410,16.808849,14.872310,14.24822,14.764940,13.77120,0.012253,0.009378,0.001016,0.002266,0.013753
865567,6917508038136902016,1237663457238712450,17.050690,15.246930,14.60410,14.417840,14.32157,0.012651,0.009405,0.007891,0.009961,0.013831
865568,6917511371031526272,1237663543139500162,18.480009,16.217600,15.25354,14.940000,14.78112,0.019448,0.010752,0.014372,0.013018,0.016123


## 5. Concatenate tables

In [12]:
#SDSS covers 3/4-sky and gaia cover all-sky, so there are not all SDSS stars in FGKM gaia table, 
#so we have to complete the missing rows with some value (for example Nan?)


In [13]:
fgkm = fgkm.set_index("source_id")
fgkm_new = fgkm_new.set_index("source_id")
fgkm_distance = fgkm_distance.set_index("source_id")
fgkm_sdss = fgkm_sdss.set_index("source_id")

In [14]:
fgkm_master = fgkm_new.join(fgkm).join(fgkm_distance).join(fgkm_sdss)

In [16]:
fgkm_master.to_csv("data/fgkm_master_nan.csv")
# This master file has nan values

In [17]:
fgkm

,teff_gspphot,logg_gspphot,mh_gspphot,ag_gspphot,ebpminrp_gspphot,alphafe_gspspec,teff_gspspec,logg_gspspec,mh_gspspec,radius_flame,lum_flame,mass_flame,age_flame,evolstage_flame,radius_flame_spec,lum_flame_spec,mass_flame_spec,age_flame_spec,evolstage_flame_spec,spectraltype_esphs
source_id,,,,,,,,,,,,,,,,,,,,
8044474553216,4806.233887,4.5114,-0.0697,0.0711,0.0375,NaN,NaN,NaN,NaN,0.826414,0.328582,NaN,NaN,393,NaN,NaN,NaN,NaN,<NA>,
12545600306304,5249.593262,4.4557,-0.0848,0.1124,0.0597,NaN,NaN,NaN,NaN,0.932639,0.596999,0.842317,13.142073,349,NaN,NaN,NaN,NaN,<NA>,
14401026146688,5172.659180,4.4139,-0.2540,0.1672,0.0892,NaN,NaN,NaN,NaN,0.975303,0.612476,NaN,NaN,352,NaN,NaN,NaN,NaN,<NA>,
15637976759168,5741.219238,4.1478,-0.0963,0.2220,0.1191,NaN,NaN,NaN,NaN,1.659690,2.649110,1.061862,8.610679,445,NaN,NaN,NaN,NaN,<NA>,
16531329957376,5654.772949,4.3989,-0.1923,0.0009,0.0005,NaN,NaN,NaN,NaN,1.031082,0.979605,0.939067,9.112502,312,NaN,NaN,NaN,NaN,<NA>,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6917508003777163904,4969.641602,4.5520,-0.0460,0.0688,0.0364,NaN,NaN,NaN,NaN,0.775331,0.330749,0.798405,8.977385,246,NaN,NaN,NaN,NaN,<NA>,
6917508038136902016,5210.476562,4.4257,0.1178,0.0456,0.0241,NaN,NaN,NaN,NaN,1.023632,0.692772,NaN,NaN,359,NaN,NaN,NaN,NaN,<NA>,
6917511371031526272,4595.608398,4.5494,-0.0827,0.1113,0.0586,NaN,NaN,NaN,NaN,0.716367,0.205604,0.729048,9.092461,244,NaN,NaN,NaN,NaN,<NA>,


## 6. Filling NaN values
- Change ```np.nan``` Radial Velocity values to ```0```
- Change ```np.nan``` Radial Velocity Error values to ```-99```

In [18]:
fgkm_master = pd.read_csv("data/fgkm_master_nan.csv")

In [19]:
fgkm_master["radial_velocity"] = fgkm_master["radial_velocity"].fillna(0)
fgkm_master["radial_velocity_error"] = fgkm_master["radial_velocity_error"].fillna(-99)

In [20]:
fgkm_master.to_csv("data/fgkm_master.csv",index=False)
# This master file does not has nan values